In [176]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
import proj1_helpers
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
yb, input_data, ids = load_csv_data("../train.csv",sub_sample = False)

In [166]:
def ultimate_split(yb, input_data, ids):
    """
    The ultimate split. THE real BADA$$.
    This is some real shit
    """
    
    mask_0 = (input_data[:,22] == 0)
    mask_1 = (input_data[:,22] == 1)
    mask_2 = (input_data[:,22] == 2)
    mask_3 = (input_data[:,22] == 3)

    
    return [[yb[mask_0], input_data[mask_0], ids[mask_0]], [yb[mask_1], input_data[mask_1], ids[mask_1]],
           [yb[mask_2], input_data[mask_2], ids[mask_2]],[yb[mask_3], input_data[mask_3], ids[mask_3]]]

def remove_features_0(input_data_0):
    """
    Remove the useless features if the jet num feature is equal to 0
    """
    useless_features_index = [4, 5, 6, 12, 23, 24, 25 , 26, 27, 28]#, 30, 33]
    
    mask = np.ones(int(input_data_0.shape[1]), dtype=bool)
    mask[(useless_features_index)] = False
    return input_data_0[:,mask]

def remove_features_1(input_data_1):
    """
    Remove the useless features if the jet num feature is equal to 1
    """
    useless_features_index = [4, 5, 6, 12, 26, 27, 28]#, 30, 33]
    
    mask = np.ones(int(input_data_1.shape[1]), dtype=bool)
    mask[(useless_features_index)] = False
    return input_data_1[:,mask]

    # No need for jet_num equal to 2 or 3 : all the features are kept

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x
def split_1st_coordinate(y,x,id):
    y1,y2,x1,x2,id1,id2 = [],[],[],[],[],[]
    
    for i in range(len(y)):
        if x[i][0] == -999:
            y1.append(y[i])
            x1.append(standardize(x[i][1:]))
            id1.append(id[i])
        else:
            y2.append(y[i])
            x2.append(standardize(x[i]))
            id2.append(id[i])
            
    return [[y1, np.array(x1), id1], 
                     [y2, np.array(x2), id2]]

D contains all the datat structures, to acess data you should use D[num_jet_case*2 + value of first column is != -999] = y,x,id

In [167]:
data_splitted = ultimate_split(yb, input_data, ids)
data_splitted[0][1] = remove_features_0(data_splitted[0][1])
data_splitted[1][1] = remove_features_1(data_splitted[1][1])

D = []

for d in data_splitted:
    s = split_1st_coordinate(d[0],d[1],d[2])
    D.append(s[0])
    D.append(s[1])
D = np.array(D)

In [177]:
def batch_ridge_in(lambda_, degree, batch_size,d,y):
    print(d.shape,y.shape)
    d_train, d_test, y_train, y_test = split_data(d,y,0.9,23)
    x = []
    xt = []
    for i in range(1,degree+1):
        x.append(np.power(d_train,i))
        xt.append(np.power(d_test,i))
        
    X_train = np.concatenate(x,axis = 1)
    #y_train = y_in_train_ref
    X_test = np.concatenate(xt,axis = 1)
    #y_test = y_in_test_ref
    
    #print("X_train shape : ", X_train.shape,"    X_test.shape : ", X_test.shape)
    #print("y_train shape : ", y_train.shape,"    y_test.shape : ", y_test.shape)
    
    num_features = X_train[0].shape[0]
    #print("number of features : ", num_features)

    #print("Splitting dataset into batch")
    X_batch = np.array_split(X_train, 1)
    y_batch = np.array_split(y_train, 1)

    #print(X_batch[0].shape)
    
    w = []
    i=0
    for y,batch in zip(y_batch,X_batch):
        w.append(ridge_regression(y, batch,lambda_))
    
    
    wp =  np.sum(np.array(w),axis=0)/(len(X_batch))
    
    
    

    y_pred_ridge_train = proj1_helpers.predict_labels(wp,X_train)
    y_pred_ridge_test = proj1_helpers.predict_labels(wp,X_test)
    s_tr = 0
    s_te = 0
    tot_tr = 0
    for i,y in enumerate(y_train):
        if y == y_pred_ridge_train[i]:
            s_tr += 1
        tot_tr += 1
    tot_te = 0
    for i,y in enumerate(y_test):
        if y == y_pred_ridge_test[i]:
            s_te += 1
        tot_te += 1

    print("Ridge precision on train : ",s_tr/tot_tr,"     Ridge precision on test : ",s_te/tot_te)
    
    return wp
    

In [185]:

for seti in range(8):
    batch_ridge_in(10,3,1,np.array(D[seti][1]),np.array(D[seti][0]))

(26123, 19) (26123,)
Ridge precision on train :  0.9407060825180774      Ridge precision on test :  0.939150401836969
(73790, 20) (73790,)
Ridge precision on train :  0.7764978693288762      Ridge precision on test :  0.782355332700908
(7562, 22) (7562,)
Ridge precision on train :  0.912417340191036      Ridge precision on test :  0.9141347424042272
(69982, 23) (69982,)
Ridge precision on train :  0.6996173570646047      Ridge precision on test :  0.7041005857979712
(2952, 29) (2952,)
Ridge precision on train :  0.8919427710843374      Ridge precision on test :  0.918918918918919
(47427, 30) (47427,)
Ridge precision on train :  0.746860650360791      Ridge precision on test :  0.7389837655492304
(1477, 29) (1477,)
Ridge precision on train :  0.9307750188111362      Ridge precision on test :  0.918918918918919
(20687, 30) (20687,)
Ridge precision on train :  0.7398216779460737      Ridge precision on test :  0.7472208796520058
